In [51]:
import openpyxl
import pandas as pd
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.styles import Border, Side
from math import sqrt

# エクセルファイルを開く
file_path = 'input_files/sample.xlsx'
wb = openpyxl.load_workbook(file_path)
sheet = wb.active

# 黒い枠線の定義
black_color = 'FF000000'

# 1. topとleftに線が引かれているセルの位置を特定しリストに格納します
top_left_cells = []
for row in sheet.iter_rows():
    for cell in row:
        if (cell.border.top and cell.border.top.color and cell.border.top.color.rgb == black_color and
            cell.border.left and cell.border.left.color and cell.border.left.color.rgb == black_color and
            (not cell.border.bottom or not cell.border.bottom.color or cell.border.bottom.color.rgb != black_color) and
            (not cell.border.right or not cell.border.right.color or cell.border.right.color.rgb != black_color)):
            top_left_cells.append(cell.coordinate)

# 2. bottomとrightだけに線が引かれているセルの位置を特定します
bottom_right_cells = []
for row in sheet.iter_rows():
    for cell in row:
        if (cell.border.bottom and cell.border.bottom.color and cell.border.bottom.color.rgb == black_color and
            cell.border.right and cell.border.right.color and cell.border.right.color.rgb == black_color and
            (not cell.border.top or not cell.border.top.color or cell.border.top.color.rgb != black_color) and
            (not cell.border.left or not cell.border.left.color or cell.border.left.color.rgb != black_color)):
            bottom_right_cells.append(cell.coordinate)

# 3. top_leftセルから見てbottom_rightセルが右下に位置しているかを確認する関数
def is_bottom_right(top_left, bottom_right):
    col1, row1 = ''.join([i for i in top_left if i.isalpha()]), ''.join([i for i in top_left if i.isdigit()])
    col2, row2 = ''.join([i for i in bottom_right if i.isalpha()]), ''.join([i for i in bottom_right if i.isdigit()])
    col1_idx, row1_idx = column_index_from_string(col1), int(row1)
    col2_idx, row2_idx = column_index_from_string(col2), int(row2)
    return col2_idx > col1_idx and row2_idx > row1_idx

# 各範囲をデータフレームとして読み込む
dataframes = []
for top_left in top_left_cells:
    min_distance = float('inf')
    closest_bottom_right = None
    for bottom_right in bottom_right_cells:
        if is_bottom_right(top_left, bottom_right):
            distance = calculate_distance(top_left, bottom_right)
            if distance < min_distance:
                min_distance = distance
                closest_bottom_right = bottom_right
    
    if closest_bottom_right:
        start_col_letter, start_row = ''.join([i for i in top_left if i.isalpha()]), ''.join([i for i in top_left if i.isdigit()])
        end_col_letter, end_row = ''.join([i for i in closest_bottom_right if i.isalpha()]), ''.join([i for i in closest_bottom_right if i.isdigit()])
        start_col = column_index_from_string(start_col_letter)
        end_col = column_index_from_string(end_col_letter)
        usecols = f"{get_column_letter(start_col)}:{get_column_letter(end_col)}"
        nrows = int(end_row) - int(start_row) + 1
        skiprows = int(start_row) - 1

        print(f"Reading range: usecols={usecols}, nrows={nrows}, skiprows={skiprows}")  # デバッグ用出力

        if nrows > 0:
            df = pd.read_excel(file_path, sheet_name=sheet.title, usecols=usecols, nrows=nrows, skiprows=skiprows, index_col=0)
            dataframes.append(df)

# 読み込んだデータフレームを表示
for i, df in enumerate(dataframes):
    print(f"DataFrame {i+1}:")
    print(df)
    print("\n")


Reading range: usecols=B:F, nrows=9, skiprows=2
Reading range: usecols=H:L, nrows=9, skiprows=2
Reading range: usecols=A:E, nrows=12, skiprows=13
Reading range: usecols=G:I, nrows=12, skiprows=13
Reading range: usecols=K:L, nrows=12, skiprows=13
DataFrame 1:
     量  割合  合計  hoge
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN


DataFrame 2:
    量.1  割合.1  合計.1  hoge.1
素材  NaN   NaN   NaN     NaN
素材  NaN   NaN   NaN     NaN
素材  NaN   NaN   NaN     NaN
素材  NaN   NaN   NaN     NaN
素材  NaN   NaN   NaN     NaN
素材  NaN   NaN   NaN     NaN
素材  NaN   NaN   NaN     NaN
素材  NaN   NaN   NaN     NaN


DataFrame 3:
     量  割合  合計  hoge
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN
素材 NaN NaN NaN   NaN


DataFrame 4:
   

In [52]:
# 新しいエクセルファイルにデータフレームを順番に書き込む
output_file = 'output_files/output.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    start_row = 0
    for i, df in enumerate(dataframes):
        df.to_excel(writer, sheet_name='Summary', startrow=start_row, index=True)
        start_row += len(df) + 2  # 2行の空白を追加

print(f"DataFrames have been written to {output_file}")

DataFrames have been written to output_files/output.xlsx
